In [2]:
pip install scikit-learn

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.9 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.9 MB 2.6 MB/s eta 0:00:04
   ----- ---------------------------------- 1.3/8.9 MB 3.2 MB/s eta 0:00:03
   ---------- ----------------------------- 2.4/8.9 MB 3.3 MB/s eta 0:00:02
   --------------- ------------------------ 3.4/8.9 MB 3.7 MB/s eta 0:00:02
   --------------------- ------------------ 4.7/8.9 MB 4.2 MB/s eta 0:00:01
   ------------------------- -------------- 5.8/8.9 MB 4.4 MB/s eta 0:00:01
   ----------------------------- ---------- 6.6/8.9 MB 4.4 MB/s eta 0:00:01
   ---------------------------------- ----- 7.6/8.9 MB 4.4 MB/s eta 0:00:01
   -------------------------------------- - 8.7/8.9 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 8.9/8.9 MB 4.5 MB/s eta 0:00:00
   ------------------------------

In [ ]:
import re
import pandas as pd
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# --------------------------
# Step 1: Utilities
# --------------------------

def extract_url(text: str):
    """Extract the first URL from given text if present"""
    url_pattern = re.compile(r'(https?://[^\s]+)')
    urls = url_pattern.findall(text)
    return urls[0] if urls else None

def extract_features(url: str):
    """Extract phishing-related features from URL"""
    features = {}
    features["url_length"] = len(url)
    features["has_at"] = 1 if "@" in url else 0
    features["has_double_slash"] = 1 if url.count("//") > 1 else 0
    features["has_dash"] = 1 if "-" in urlparse(url).netloc else 0
    features["is_https"] = 1 if url.startswith("https") else 0
    features["domain_length"] = len(urlparse(url).netloc)
    return features

# --------------------------
# Step 2: Load Training Data
# --------------------------

# Load your converted dataset
df = pd.read_csv("phishing_Training_Dataset.csv")

# ⚠️ Adjust these two lines if your CSV has different column names
X = df.drop("Result", axis=1)   # Features
y = df["Result"]                # Labels (0 = Safe, 1 = Phishing)

# --------------------------
# Step 3: Build Model
# --------------------------

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy="mean")),
            ('scaler', StandardScaler())
        ]), X.columns)
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])

pipeline.fit(X, y)

print("✅ Model trained on Training Dataset.csv")


# --------------------------
# Step 4: Predict Function
# --------------------------

def predict_content(text: str) -> int:
    """
    Predict if given text contains phishing or not.
    Returns:
        1 -> phishing
        0 -> safe
    """
    url = extract_url(text)
    if not url:
        return 0  # No URL = safe

    features = extract_features(url)
    features_df = pd.DataFrame([features])

    prediction = pipeline.predict(features_df)[0]
    return int(prediction)


# --------------------------
# Step 5: Run Example
# --------------------------
if __name__ == "__main__":
    user_input = input("Enter a message/post/comment: ")
    result = predict_content(user_input)

    if result == 1:
        print("🚨 Phishing detected (1)")
    else:
        print("✅ Safe content (0)")


✅ Model trained on Training Dataset.csv
